## Adventures with Elephants

Description of the dataset:
---------------------------

This dataset has been recorded Angela S. Stoeger (angela.stoeger-horwath@univie.ac.at) from Vienna University and colleagues. Acoustic recordings were collected from three female and two male African elephants (Loxodonta africana) aged between 9 and 17 years located at Adventures with Elephants, Bela Bela, South Africa in 2011. The elephants were fully habituated to human presence and free to roam around in a savannah reserve of 300 ha. Vocalizations were recorded during two social contexts: spatial separation and subsequent bondings, recording distances between the elephant and the microphone ranged from 10 to several hundreds of meters. 
In 359 minutes of recordings we annotated 635 rumbles by manually tagging the beginning and the end of each rumble.

In [14]:
# annotations are in matlab file, which apparently is a hdf5 format
import scipy.io as sio
filepath = '../../data/AdventuresWithElephants_Dataset/anno/groundTruth.mat'

annos = sio.loadmat(filepath)
data = annos['variable']


array([array(['ds_ID'],
      dtype='<U5'),
       array(['P'],
      dtype='<U1'), array(['L'],
      dtype='<U1'),
       array(['singleType'],
      dtype='<U10')], dtype=object)

In [86]:
# r=[(x['SHAPE@XY'][i][0],x['SHAPE@XY'][i][1],x['PrimeKey'][i]) for i in range(x.shape[0])]
# unpack nd array
u = [(data[i,0][0],data[i,1][0][0],data[i,2][0][0],data[i,3][0]) for i in range(1,data.shape[0])]

In [87]:
# build dataframe from list
import pandas as pd
df = pd.DataFrame(data=u)  # 1st row as the column names
df.columns=['filename','start','duration','type']

list

Data format:
------------

Folder "wavs" contains all recorded signals (sampled at 44100Hz, 16bit, mono)

Folder "anno" contains the annotation in binary Matlab format (*.mat). The annotations include:

- groundTruth.mat: a cell that contains one annotated rumble in each row. The first column provides the name of the wav-file. The second column gives the start position of the rumble in samples; the third column contains the length of the annotated rumble in samples. The fourth column contains the type of rumble. There are two types (the usual "rumble" and the "snumble", which is a mixture of snort and rumble).

In [128]:
#df['stop'] = df.start + df.duration
df.head(10)


,filename,start,duration,type,stop
0,EFAF2011A008.WAV,1688633,51548,rumble,1740181
1,EFAF2011A008.WAV,1829197,24136,rumble,1853333
2,EFAF2011A008.WAV,4925581,41697,rumble,4967278
3,EFAF2011A008.WAV,7701919,41816,rumble,7743735
4,EFAF2011A008.WAV,7848265,44664,rumble,7892929
5,EFAF2011A008.WAV,7917705,30799,rumble,7948504
6,EFAF2011A008.WAV,8041171,57017,rumble,8098188
7,EFAF2011A008.WAV,10693742,76558,rumble,10770300
8,EFAF2011A008.WAV,10795225,65877,rumble,10861102
9,EFAF2011A008.WAV,10902049,35610,rumble,10937659


In [124]:
# issue with EFAF2011A011.WAV
df = df[df['filename'] != "EFAF2011A011.WAV"]


In [129]:
# now we open the source file, cut out the piece that is annotated and save that as a 5 second sample
import sys, csv, os, shutil

# we will also resample the original recordings
import librosa
import resampy

from tqdm import tqdm

# this is the sample reate we want
sr_target = 44100

# source dataset location
source_path = '../../data/AdventuresWithElephants_Dataset/wavs/all/'
dest_path = "../../data/AdventuresWithElephants_Dataset/wavs/samples/"

fileList = []

# for each annotation do
for index,row in tqdm(df.iterrows(), total=len(df)):
    # map mat file contents
    source_filename = row.filename
    start = row.start
    duration = row.duration
    category = row.type
    
    # read source wav 
    src_file_path = os.path.normcase("%s/%s" % (source_path,source_filename))
    
    # check if sourcefile exists
    if os.path.exists(src_file_path):
        # resample
        # Load audio file at its native sampling rate
        x, sr_orig = librosa.load(src_file_path, mono=True, sr=None)

        # We can resample this to any sampling rate we like, say 16000 Hz
        # define 5 seconds
        s5 = 5 * sr_orig
        # clip from start to next 5 seconds resample in target sample rate
        y = resampy.resample(x[start:start+s5+1], sr_orig, sr_target)

        # check if previous 5 secs is background
        # if so create a background sample
        if (start-s5 > 0):
            # clip from start to next 5 seconds resample in target sample rate
            yb = resampy.resample(x[start-s5:start+1], sr_orig, sr_target)
            # make destination path
            b_dst_fname = source_filename.split('.')[0] + '-' + str(round((start-s5)/sr_orig)) + '.' + source_filename.split('.')[1]
            # add meta data to list
            fileList.append([b_dst_fname,'background'])
            # path
            b_dst_file_path = "%s/%s" % (dest_path,b_dst_fname)
            # write it back
            librosa.output.write_wav(b_dst_file_path, yb, sr_target)
            
        # make destination path
        dest_filename = source_filename.split('.')[0] + '-' + str(round(start/sr_orig)) + '.' + source_filename.split('.')[1]
        dst_file_path = "%s/%s" % (dest_path,dest_filename)

        # create meta data file
        fileList.append([dest_filename,category])

        # write it back
        librosa.output.write_wav(dst_file_path, y, sr_target)
    else:
        print('File %s not found.' % source_filename)

  6%|▌         | 36/618 [00:34<08:52,  1.09it/s]

File EFAF2011A012.WAV not found.
File EFAF2011A012.WAV not found.
File EFAF2011A012.WAV not found.
File EFAF2011A012.WAV not found.
File EFAF2011A012.WAV not found.
File EFAF2011A012.WAV not found.


100%|██████████| 618/618 [21:29<00:00,  1.73s/it]


In [130]:
# save the metadata for later
fileList
md = pd.DataFrame(fileList, columns=["filename", "category"])  
md.to_csv('./dataset/meta-data-elephant-adventures.csv',index=False)

In [132]:
md.tail()

,filename,category
1217,EFAF2011A033-427.WAV,rumble
1218,EFAF2011A033-28492206.WAV,background
1219,EFAF2011A033-646.WAV,rumble
1220,EFAF2011A033-28575657.WAV,background
1221,EFAF2011A033-648.WAV,rumble


In [133]:
# copy all wavs to audio dir in serval
import glob
import shutil
import os

src_dir = "../../data/AdventuresWithElephants_Dataset/wavs/samples/"
dst_dir = "./dataset/audio/"
for jpgfile in glob.iglob(os.path.join(src_dir, "*.WAV")):
    shutil.copy(jpgfile, dst_dir)
